# Building the Classifier

## 1. Loading the corpus

In [1]:
import re
from nltk import sent_tokenize

cg_sents = []
smg_sents = []

def remove_duplicate_punctuation(s):
    return re.sub(r'([\.\?!;])\1+', r'\1 ', s)

def fix_new_sentence_spacing(s):
    return re.sub(r'([a-zα-ωίϊΐόάέύϋΰήώ])([\.\?!;])([A-ZΑ-ΩΆΈΊΌΎΏΉ])', r'\1\2 \3', s)

with open('./Data/cg_twitter.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        cg_sents += sent_tokenize(line)

with open('./Data/cg_fb.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        cg_sents += sent_tokenize(line)

with open('./Data/cg_other.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        cg_sents += sent_tokenize(line)

with open('./Data/smg_twitter.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        smg_sents += sent_tokenize(line)

with open('./Data/smg_fb.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        smg_sents += sent_tokenize(line)

with open('./Data/smg_other.txt', 'r', encoding='utf-8') as in_file:
    text = remove_duplicate_punctuation(in_file.read())
    text = fix_new_sentence_spacing(text)
    lines = [p for p in text.split('\n') if p]
    for line in lines:
        smg_sents += sent_tokenize(line)

cg_sents[:3]

['Πρασινο αυκουι μες το πασχαλινο ποτήρι που έπιασε ο μιτσης #αισχος 🤣🤣🤣   @HARRIS_APOEL https://t.co/y9X7CmBEa5',
 '@HARRIS_APOEL @pirpoitis @vassrules Καμνουν πολλα ανακαινιση στα Περβολια .',
 '@MUFCChristian Ελα συγγενη τζιαι εχουμε νεοτερα π το Νικολη.']

## 2. Cleaning the text

In [2]:
import unicodedata
from string import punctuation
from nltk.tokenize import WhitespaceTokenizer

punctuation += '´΄’…“”–—―»«'

def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def get_clean_sent_el(sentence):
    sentence = re.sub(r'^RT', '', sentence)
    sentence = re.sub(r'\&\w*;', '', sentence)
    sentence = re.sub(r'\@\w*', '', sentence)
    sentence = re.sub(r'\$\w*', '', sentence)
    sentence = re.sub(r'https?:\/\/.*\/\w*', '', sentence)
    sentence = ''.join(c for c in sentence if c <= '\uFFFF')
    sentence = strip_accents(sentence)
    sentence = re.sub(r'#\w*', '', sentence)
    sentence = sentence.lower()
    tokens = WhitespaceTokenizer().tokenize(sentence)
    new_tokens = []
    for token in tokens:
        if token == 'ο,τι' or token == 'ό,τι' or token == 'o,ti' or token == 'ó,ti':
            new_tokens.append(token)
        else:
            token = re.sub(r'(?<=[.,!\?;\'΄´])(?=[^\s])', r' ', token)
            new_token = token.translate(str.maketrans({key: None for key in punctuation}))
            if new_token != '':
                new_tokens.append(new_token)
    sentence =' '.join(new_tokens)
    sentence = sentence.replace('\ufeff', '')
    sentence = sentence.strip(' ')
    sentence = sentence.replace('  ', ' ')
    return sentence

cg_sents_clean = []
smg_sents_clean = []

for sent in cg_sents:
    cg_sents_clean.append(get_clean_sent_el(sent))
for sent in smg_sents:
    smg_sents_clean.append(get_clean_sent_el(sent))

cg_sents_clean = list(filter(None, cg_sents_clean))
smg_sents_clean = list(filter(None, smg_sents_clean))
cg_sents_clean[:3]

['πρασινο αυκουι μες το πασχαλινο ποτηρι που επιασε ο μιτσης',
 'καμνουν πολλα ανακαινιση στα περβολια',
 'ελα συγγενη τζιαι εχουμε νεοτερα π το νικολη']

## 3. Building the feature extractor

In [3]:
from nltk import ngrams

def get_word_ngrams(tokens, n):
    ngrams_list = []
    ngrams_list.append(list(ngrams(tokens, n)))
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = '%s'
    for _ in range(1, n):
        format_string += (' %s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

def get_char_ngrams(word, n):
    ngrams_list = []
    word = word.replace('ς', 'σ')
    ngrams_list.append(list(ngrams(word, n, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))

    # Removing redundant ngrams:
    if n > 2:
        redundant_combinations = n - 2
        ngrams_list = [ngram_list[redundant_combinations : -redundant_combinations] for ngram_list in ngrams_list]

    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = ''
    for _ in range(0, n):
        format_string += ('%s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

In [4]:
# Feature extractor
def get_ngram_features(sent): # The reason I do not use NLTK's everygrams to extract the features quickly is because the behavior of my n-gram extractor is modified to remove redundant n-grams. Also, I need to label word and char n-grams to avoid ambiguity
    sentence_tokens = WhitespaceTokenizer().tokenize(sent)

    features = {}

    # Word unigrams
    ngrams = get_word_ngrams(sentence_tokens, 1)
    for ngram in ngrams:
        features[f'word({ngram})'] = features.get(f'word({ngram})', 0) + 1 # The second parameter to .get() is a default value if the key doesn't exist.

    # Word bigrams
    ngrams = get_word_ngrams(sentence_tokens, 2)
    for ngram in ngrams:
        features[f'word_bigram({ngram})'] = features.get(f'word_bigram({ngram})', 0) + 1

    # Char unigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 1)
        for ngram in ngrams:
            features[f'char({ngram})'] = features.get(f'char({ngram})', 0) + 1

    # Char bigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 2)
        for ngram in ngrams:
            features[f'char_bigram({ngram})'] = features.get(f'char_bigram({ngram})', 0) + 1

    # Char trigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 3)
        for ngram in ngrams:
            features[f'char_trigram({ngram})'] = features.get(f'char_trigram({ngram})', 0) + 1

    return features

get_ngram_features('αυτη ειναι η σπαρτη')

{'word(αυτη)': 1,
 'word(ειναι)': 1,
 'word(η)': 1,
 'word(σπαρτη)': 1,
 'word_bigram(αυτη ειναι)': 1,
 'word_bigram(ειναι η)': 1,
 'word_bigram(η σπαρτη)': 1,
 'char(α)': 3,
 'char(υ)': 1,
 'char(τ)': 2,
 'char(η)': 3,
 'char(ε)': 1,
 'char(ι)': 2,
 'char(ν)': 1,
 'char(σ)': 1,
 'char(π)': 1,
 'char(ρ)': 1,
 'char_bigram(_α)': 1,
 'char_bigram(αυ)': 1,
 'char_bigram(υτ)': 1,
 'char_bigram(τη)': 2,
 'char_bigram(η_)': 3,
 'char_bigram(_ε)': 1,
 'char_bigram(ει)': 1,
 'char_bigram(ιν)': 1,
 'char_bigram(να)': 1,
 'char_bigram(αι)': 1,
 'char_bigram(ι_)': 1,
 'char_bigram(_η)': 1,
 'char_bigram(_σ)': 1,
 'char_bigram(σπ)': 1,
 'char_bigram(πα)': 1,
 'char_bigram(αρ)': 1,
 'char_bigram(ρτ)': 1,
 'char_trigram(_αυ)': 1,
 'char_trigram(αυτ)': 1,
 'char_trigram(υτη)': 1,
 'char_trigram(τη_)': 2,
 'char_trigram(_ει)': 1,
 'char_trigram(ειν)': 1,
 'char_trigram(ινα)': 1,
 'char_trigram(ναι)': 1,
 'char_trigram(αι_)': 1,
 'char_trigram(_η_)': 1,
 'char_trigram(_σπ)': 1,
 'char_trigram(σπα)': 1,

In [5]:
# from nltk import everygrams

# def sent_process(sent):
#     return [''.join(ng) for ng in everygrams(sent.replace(' ', '_ _'), 1, 4)
#             if ' ' not in ng and '\n' not in ng and ng != ('_',)]

# sent_process('αυτη ειναι η σπαρτη')

## 4. Creating the training and test sets

In [6]:
import random

all_sents_labeled = ([(sentence, 'CG') for sentence in cg_sents_clean] + [(sentence, 'SMG') for sentence in smg_sents_clean])
random.shuffle(all_sents_labeled)
all_sents_labeled[0]

('απο τη στιγμη που δεχθηκα μεσα μου πως αυτος ο τοπος δεν εχει σωτηρια εχω πραγματικα ηρεμησει',
 'SMG')

In [7]:
NO_ALL_SENTENCES = len(all_sents_labeled)
NO_TRAIN_SENTENCES = round(NO_ALL_SENTENCES * .8)

train_set = all_sents_labeled[:NO_TRAIN_SENTENCES]
test_set = all_sents_labeled[NO_TRAIN_SENTENCES:]

train_set_sents = [sent[0] for sent in train_set]
train_set_labels = [sent[1] for sent in train_set]
test_set_sents = [sent[0] for sent in test_set]
test_set_labels = [sent[1] for sent in test_set]

print(train_set_sents[0], train_set_labels[0])

απο τη στιγμη που δεχθηκα μεσα μου πως αυτος ο τοπος δεν εχει σωτηρια εχω πραγματικα ηρεμησει SMG


In [8]:
print('DATASET\t', 'SENTENCES')
print('All\t', NO_ALL_SENTENCES)
print('Training', NO_TRAIN_SENTENCES)
print('Testing\t', NO_ALL_SENTENCES - NO_TRAIN_SENTENCES)

DATASET	 SENTENCES
All	 1039
Training 831
Testing	 208


## 5. Vectorization

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=get_ngram_features)

train_set_vectors = count_vect.fit_transform(train_set_sents)
test_set_vectors = count_vect.transform(test_set_sents) # Unlike fit_transform(), transform() does not change the count vectorizer's vocabulary so it should be used for the test set.
train_set_vectors

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 125935 stored elements and shape (831, 16129)>

In [10]:
train_set_vectors.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(16129,))

In [11]:
count_vect.vocabulary_ # The numbers are not counts but indices.

{'word(απο)': 3761,
 'word(τη)': 6773,
 'word(στιγμη)': 6571,
 'word(που)': 6266,
 'word(δεχθηκα)': 4193,
 'word(μεσα)': 5577,
 'word(μου)': 5660,
 'word(πως)': 6390,
 'word(αυτος)': 3903,
 'word(ο)': 5818,
 'word(τοπος)': 6799,
 'word(δεν)': 4183,
 'word(εχει)': 4830,
 'word(σωτηρια)': 6683,
 'word(εχω)': 4842,
 'word(πραγματικα)': 6285,
 'word(ηρεμησει)': 4902,
 'word_bigram(απο τη)': 7669,
 'word_bigram(τη στιγμη)': 14786,
 'word_bigram(στιγμη που)': 14161,
 'word_bigram(που δεχθηκα)': 13298,
 'word_bigram(δεχθηκα μεσα)': 8388,
 'word_bigram(μεσα μου)': 11424,
 'word_bigram(μου πως)': 11716,
 'word_bigram(πως αυτος)': 13638,
 'word_bigram(αυτος ο)': 7865,
 'word_bigram(ο τοπος)': 12328,
 'word_bigram(τοπος δεν)': 15412,
 'word_bigram(δεν εχει)': 8327,
 'word_bigram(εχει σωτηρια)': 9610,
 'word_bigram(σωτηρια εχω)': 14344,
 'word_bigram(εχω πραγματικα)': 9671,
 'word_bigram(πραγματικα ηρεμησει)': 13487,
 'char(α)': 4,
 'char(π)': 19,
 'char(ο)': 18,
 'char(τ)': 22,
 'char(η)': 10,
 '

In [12]:
len(count_vect.vocabulary_) # This is the same as the length of each vector.

16129

## 6. Building the classifiers

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def show_performance_header_and_accuracy(predictions):
    print('\t\t\tPERFORMANCE\n')
    print('Accuracy:', round(accuracy_score(test_set_labels, predictions), 2), '\n')

def show_confusion_matrix(cm):
    print('\t         Predicted')
    print('\t        CG       SMG')
    print('\t     -------- --------')
    print('\tCG  | {:^6} | {:^6}'.format(cm[0][0], cm[0][1]))
    print('Actual\t     -------- --------')
    print('\tSMG | {:^6} | {:^6}'.format(cm[1][0], cm[1][1]))

def show_most_informative_features(vectorizer, clf, n=10):
    print("\t\t    CG\t\t\t\t\t\t    SMG\n")
    feature_names = vectorizer.get_feature_names_out()
    coefs = clf.coef_[0] if hasattr(clf, 'coef_') else clf.feature_log_prob_[1] - clf.feature_log_prob_[0]
    coefs_with_fns = sorted(zip(coefs, feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%17s\t\t\t%.4f\t%17s" % (coef_1, fn_1, coef_2, fn_2))

### 6.1 Multinomial Naive Bayes classifier

In [14]:
clf_multinomialNB = MultinomialNB() # There are no params for MultinomialDB that prevent overfitting, so any overfitting is caused by the small dataset size.
clf_multinomialNB.fit(train_set_vectors, train_set_labels)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [15]:
clf_multinomialNB_predictions = clf_multinomialNB.predict(test_set_vectors)

show_performance_header_and_accuracy(clf_multinomialNB_predictions)

print(classification_report(test_set_labels, clf_multinomialNB_predictions))

cmatrix = confusion_matrix(test_set_labels, clf_multinomialNB_predictions)
show_confusion_matrix(cmatrix)

			PERFORMANCE

Accuracy: 0.96 

              precision    recall  f1-score   support

          CG       0.92      1.00      0.96       105
         SMG       1.00      0.91      0.95       103

    accuracy                           0.96       208
   macro avg       0.96      0.96      0.96       208
weighted avg       0.96      0.96      0.96       208

	         Predicted
	        CG       SMG
	     -------- --------
	CG  |  105   |   0   
Actual	     -------- --------
	SMG |   9    |   94  


In [16]:
show_most_informative_features(count_vect, clf_multinomialNB, n=20)

		    CG						    SMG

	-4.6252	char_trigram(ιαι)			2.7525	         word(δε)
	-4.5844	      word(τζιαι)			2.7525	char_trigram(δε_)
	-4.1309	  char_bigram(θκ)			2.6190	char_trigram(χισ)
	-4.0805	char_trigram(σιε)			2.6190	char_trigram(χετ)
	-3.9119	char_trigram(ζαι)			2.6190	  char_bigram(σθ)
	-3.8912	char_trigram(θκι)			2.5449	    word(γυναικα)
	-3.8487	       word(τζαι)			2.5449	char_trigram(ισθ)
	-3.8378	         word(εν)			2.5449	char_trigram(_πη)
	-3.8042	  char_bigram(φκ)			2.4649	word_bigram(δεν ειναι)
	-3.8042	char_trigram(_τζ)			2.4649	       word(πολυ)
	-3.7577	char_trigram(τζι)			2.4649	char_trigram(σθη)
	-3.6310	char_trigram(λαλ)			2.4649	char_trigram(λυ_)
	-3.6036	char_trigram(αμν)			2.4136	        word(απο)
	-3.4540	  char_bigram(τζ)			2.3779	        word(ολα)
	-3.4540	char_trigram(υλλ)			2.3779	        word(μην)
	-3.3522	        word(μεν)			2.3779	      word(εχετε)
	-3.3159	        word(τες)			2.3779	       word(αυτη)
	-3.2972	char_trigram(ζια)			2.3153	char_trigram(ολυ)


### 6.2 Linear Support Vector classifier

In [17]:
clf_linearSVC = LinearSVC(max_iter=1500) # n_samples < n_features in training set so the dual param is kept at its default value of True. Default max_iter = 1000
clf_linearSVC.fit(train_set_vectors, train_set_labels)

,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",None
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo rand

In [18]:
clf_linearSVC_predictions = clf_linearSVC.predict(test_set_vectors)

show_performance_header_and_accuracy(clf_linearSVC_predictions)

print(classification_report(test_set_labels, clf_linearSVC_predictions))

cmatrix = confusion_matrix(test_set_labels, clf_linearSVC_predictions)
show_confusion_matrix(cmatrix)

			PERFORMANCE

Accuracy: 0.95 

              precision    recall  f1-score   support

          CG       0.94      0.96      0.95       105
         SMG       0.96      0.94      0.95       103

    accuracy                           0.95       208
   macro avg       0.95      0.95      0.95       208
weighted avg       0.95      0.95      0.95       208

	         Predicted
	        CG       SMG
	     -------- --------
	CG  |  101   |   4   
Actual	     -------- --------
	SMG |   6    |   97  


In [19]:
show_most_informative_features(count_vect, clf_linearSVC, n=20)

		    CG						    SMG

	-0.3551	char_trigram(εν_)			0.1517	        word(και)
	-0.3151	         word(εν)			0.1461	        word(δεν)
	-0.2320	char_trigram(_τζ)			0.1461	char_trigram(δεν)
	-0.2197	  char_bigram(τζ)			0.1392	  char_bigram(λυ)
	-0.2097	  char_bigram(_ε)			0.1378	          char(ι)
	-0.2007	  char_bigram(αμ)			0.1329	  char_bigram(να)
	-0.1920	  char_bigram(θκ)			0.1298	char_trigram(ινα)
	-0.1762	char_trigram(θκι)			0.1233	  char_bigram(τι)
	-0.1708	char_trigram(καμ)			0.1184	  char_bigram(η_)
	-0.1650	char_trigram(_λα)			0.1179	char_trigram(τασ)
	-0.1500	char_trigram(ιαν)			0.1133	         word(δε)
	-0.1404	  char_bigram(φκ)			0.1133	char_trigram(δε_)
	-0.1359	char_trigram(τι_)			0.1104	          char(ν)
	-0.1323	char_trigram(_εν)			0.1082	char_trigram(θελ)
	-0.1312	char_trigram(σιε)			0.1068	  char_bigram(λο)
	-0.1290	  char_bigram(λλ)			0.1067	  char_bigram(_β)
	-0.1281	  char_bigram(ν_)			0.1050	char_trigram(με_)
	-0.1201	char_trigram(_ου)			0.1033	       word(καλα)
	-0.1

### 6.3 Logistic Regression classifier

In [20]:
clf_logisticRegression = LogisticRegression() # Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
clf_logisticRegression.fit(train_set_vectors, train_set_labels)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [21]:
clf_logisticRegression_predictions = clf_logisticRegression.predict(test_set_vectors)

show_performance_header_and_accuracy(clf_logisticRegression_predictions)

print(classification_report(test_set_labels, clf_logisticRegression_predictions))

cmatrix = confusion_matrix(test_set_labels, clf_logisticRegression_predictions)
show_confusion_matrix(cmatrix)

			PERFORMANCE

Accuracy: 0.94 

              precision    recall  f1-score   support

          CG       0.94      0.94      0.94       105
         SMG       0.94      0.94      0.94       103

    accuracy                           0.94       208
   macro avg       0.94      0.94      0.94       208
weighted avg       0.94      0.94      0.94       208

	         Predicted
	        CG       SMG
	     -------- --------
	CG  |   99   |   6   
Actual	     -------- --------
	SMG |   6    |   97  


In [22]:
show_most_informative_features(count_vect, clf_logisticRegression, n=20)

		    CG						    SMG

	-1.3091	char_trigram(εν_)			0.6666	        word(και)
	-1.1892	         word(εν)			0.4902	        word(δεν)
	-0.8828	  char_bigram(τζ)			0.4902	char_trigram(δεν)
	-0.8754	char_trigram(_τζ)			0.4800	char_trigram(ινα)
	-0.8114	  char_bigram(αμ)			0.4508	  char_bigram(λυ)
	-0.7040	  char_bigram(_ε)			0.4376	  char_bigram(να)
	-0.6453	  char_bigram(θκ)			0.4253	char_trigram(και)
	-0.6143	char_trigram(καμ)			0.4239	char_trigram(_απ)
	-0.5972	char_trigram(_εν)			0.3943	char_trigram(τασ)
	-0.5454	char_trigram(θκι)			0.3876	  char_bigram(η_)
	-0.5354	  char_bigram(ν_)			0.3851	      word(ειναι)
	-0.5134	char_trigram(_λα)			0.3831	  char_bigram(τι)
	-0.4810	  char_bigram(φκ)			0.3773	          char(δ)
	-0.4763	  char_bigram(σι)			0.3623	char_trigram(ειν)
	-0.4586	char_trigram(σιε)			0.3526	  char_bigram(ικ)
	-0.4540	char_trigram(ιαν)			0.3443	         word(δε)
	-0.4515	  char_bigram(λλ)			0.3443	char_trigram(δε_)
	-0.4513	char_trigram(τζι)			0.3438	  char_bigram(_α)
	-0.4

**It seems that the classification algorithm with the best performance is *Multinomial Naive Bayes***.

## 7. Analyzing misclassifications made by the Multinomial Naive Bayes classifier

In [23]:
print('MISCLASSIFICATIONS\n')

misclassificationCount = 0

for i, sent in enumerate(test_set_sents):
    if test_set_labels[i] != clf_multinomialNB_predictions[i]:
        misclassificationCount += 1
        print(f'{misclassificationCount}.', sent, f'(CORRECT = {test_set_labels[i]},', f'PREDICTED = {clf_multinomialNB_predictions[i]})\n')

MISCLASSIFICATIONS

1. να βρω την φωτο να την στειλω (CORRECT = SMG, PREDICTED = CG)

2. ολοι ετσι κανουν (CORRECT = SMG, PREDICTED = CG)

3. προσπαθουσα να καθαρισω το λαιμο μου και νομιζω ειπα κατι στα εβραικα (CORRECT = SMG, PREDICTED = CG)

4. διαβασα την ατακα στο τελος και πεταχτηκα μαλακα (CORRECT = SMG, PREDICTED = CG)

5. πρεπει να γραφτει σε ολες τις γλωσσες να το μαθουν παντου ρε (CORRECT = SMG, PREDICTED = CG)

6. αν εμεις δεν καταφεραμε τοτε οσα λαχταρησαμε δεν ξεσπαμε πανω τους (CORRECT = SMG, PREDICTED = CG)

7. μολις ψηφιστηκε στη γαλλια νομος π ποινικοποιει τους πελατες της πορνειας (CORRECT = SMG, PREDICTED = CG)

8. κι εμεις που μεγαλωσαμε πλαι στην θαλασσα και νοσταλγουμε το νερο (CORRECT = SMG, PREDICTED = CG)

9. με πηρε μια τηλεφωνο και μου ειπε οτι μου δινει δωρο ενα ταμπλετ για το πασχα (CORRECT = SMG, PREDICTED = CG)



## 8. Trying the Multinomial Naive Bayes classifier with custom input

First, a more powerful version of the classifier is built by using all the data available:

In [24]:
full_set_sents = [sent[0] for sent in all_sents_labeled]
full_set_labels = [sent[1] for sent in all_sents_labeled]
full_set_vectors = count_vect.fit_transform(full_set_sents)

clf_super_multinomialNB = MultinomialNB()
clf_super_multinomialNB.fit(full_set_vectors, full_set_labels)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


Trying 2 custom sentences:

In [25]:
cgSent = 'Η Κύπρος εν που τες πιο όμορφες χώρες.'
smgSent = 'Η Κύπρος είναι από τις πιο όμορφες χώρες.'

demoSentences = [cgSent, smgSent]

cgSent = get_clean_sent_el(cgSent)
smgSent = get_clean_sent_el(smgSent)

test_vec = count_vect.transform([cgSent, smgSent])

for sentenceNumber, predictionArr in enumerate(clf_super_multinomialNB.predict_proba(test_vec)):
    print(f'SENTENCE {sentenceNumber + 1}: “{demoSentences[sentenceNumber]}”')
    if predictionArr[0] > predictionArr[1]:
        print(f'PREDICTION: Cypriot Greek (Confidence: {predictionArr[0]:.2f})\n')
    else:
        print(f'PREDICTION: Standard Modern Greek (Confidence: {predictionArr[1]:.2f})\n')

SENTENCE 1: “Η Κύπρος εν που τες πιο όμορφες χώρες.”
PREDICTION: Cypriot Greek (Confidence: 1.00)

SENTENCE 2: “Η Κύπρος είναι από τις πιο όμορφες χώρες.”
PREDICTION: Standard Modern Greek (Confidence: 1.00)

